In [1]:
import pandas as pd
import mysql.connector

A partir del ejemplo del fichero countries.csv vamos a explorar como cargar nuestra base de datos con un fichero csv, para luego reproducirlo con nuestra base de datos

In [4]:
# Leer el CSV
df = pd.read_csv("countries.csv")

In [6]:
df

,id,country,population,capital
0,1,USA,"329,500,000",Washington D.C.
1,2,Canada,"38,010,000",Ottawa
2,3,UK,"67,220,000",London
3,4,Spain,"47,350,000",Madrid
4,5,Greenland,"56,367",Nuuk


In [7]:
# Conexión a MySQL
conn = mysql.connector.connect(
    host="localhost", 
    user="root", 
    password="contraseña",
    database="data"
)

In [8]:
cursor = conn.cursor()

In [13]:
columnas = df.columns.tolist()
columnas

['id', 'country', 'population', 'capital']

### Reproduccion en nuestra base de datos, cargar nuestra base de datos con un csv a partir de la librería mysql.connector 

In [21]:
# Leer el CSV
df2 = pd.read_csv("madrid_sale_properties_details.csv")

In [22]:
df2.sample(3)

,url,property_native_id,price_eur,barrio,distrito,latitude,longitude,scrape_status,scraped_timestamp,description,...,Superficie útil,Teléfono,Terraza,Tipo de casa,Tipo suelo,Trastero,Urbanizado,Vidrios dobles,page_source,Unnamed: 63
2520,https://www.pisos.com/comprar/duplex-puente_de...,5.416724e+10,325000.0,San Diego,Puente de Vallecas,40.3968692,-3.6674685,Success,"2025-05-28T18:45:19,451",Luminoso dúplex con terraza y vistas a la Plaz...,...,53 m²,NaN,True,NaN,NaN,NaN,NaN,NaN,84.0,NaN
1961,https://www.pisos.com/comprar/piso-acacias2800...,5.257548e+10,649900.0,Acacias,Arganzuela,40.3998317,-3.70185,Success,"2025-05-28T18:28:31,465",Agencia inmobiliaria de Madrid zona Arganzuela...,...,NaN,NaN,NaN,NaN,Gres,NaN,NaN,NaN,66.0,NaN
1189,https://www.pisos.com/comprar/piso-ciudad_line...,NaN,NaN,NaN,NaN,40.4561278,-3.6597554,Success,"2025-05-28T18:05:20,975",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN


In [23]:
# Conexión a MySQL
conn2 = mysql.connector.connect(
    host="localhost", 
    user="root", 
    password="contraseña",
    database="realestate"
)

In [24]:
cursor2 = conn2.cursor()

In [25]:
columnas2 = df2.columns.tolist()
columnas2

['url',
 'property_native_id',
 'price_eur',
 'barrio',
 'distrito',
 'latitude',
 'longitude',
 'scrape_status',
 'scraped_timestamp',
 'description',
 'energy_certificate_main_classification',
 'energy_consumption_rating',
 'energy_consumption_value',
 'energy_emissions_rating',
 'energy_emissions_value',
 'Adaptado a personas con movilidad reducida',
 'Agua',
 'Aire acondicionado',
 'Amueblado',
 'Antigüedad',
 'Armarios empotrados',
 'Ascensor',
 'Balcón',
 'Baños',
 'Calefacción',
 'Calle alumbrada',
 'Calle asfaltada',
 'Carpintería exterior',
 'Carpintería interior',
 'Chimenea',
 'Cocina equipada',
 'Comedor',
 'Conservación',
 'Exterior',
 'Garaje',
 'Gas',
 'Gastos de comunidad',
 'Habitaciones',
 'Interior',
 'Jardín',
 'Lavadero',
 'Luz',
 'No se aceptan mascotas',
 'Orientación',
 'Piscina',
 'Planta',
 'Portero automático',
 'Puerta blindada',
 'Referencia',
 'Se aceptan mascotas',
 'Sistema de seguridad',
 'Soleado',
 'Superficie construida',
 'Superficie solar',
 'Super

In [17]:

# Construir query SQL
sql = f"""
    INSERT INTO countries ({', '.join(columnas)})
    VALUES ({', '.join(['%s'] * len(columnas))})
"""

In [27]:
# Construir query SQL
sql2 = f"""
    INSERT INTO staging_raw_scrape ({', '.join(columnas2)})
    VALUES ({', '.join(['%s'] * len(columnas2))})
"""

In [15]:
# Convertir DataFrame a lista de tuplas
datos = [
    tuple(row[col] if pd.notna(row[col]) else None for col in columnas)
    for _, row in df.iterrows()
]

In [28]:
# Convertir DataFrame a lista de tuplas
datos2 = [
    tuple(row[col] if pd.notna(row[col]) else None for col in columnas2)
    for _, row in df2.iterrows()
]

In [18]:
# Insertar todos los datos de golpe
cursor.executemany(sql, datos)
conn.commit()


In [29]:
# Insertar todos los datos de golpe
cursor2.executemany(sql2, datos2)
conn2.commit()

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'a personas con movilidad reducida, Agua, Aire acondicionado, Amueblado, Antigüe' at line 1

In [19]:
cursor.execute("SHOW TABLES")
for (table_name,) in cursor.fetchall():
    print(table_name)

countries


In [20]:
cursor.execute("SELECT * FROM countries")
filas = cursor.fetchall()

# Mostrar los resultados
for fila in filas:
    print(fila)

(1, 'USA', '329,500,000', 'Washington D.C.')
(2, 'Canada', '38,010,000', 'Ottawa')
(3, 'UK', '67,220,000', 'London')
(4, 'Spain', '47,350,000', 'Madrid')
(5, 'Greenland', '56,367', 'Nuuk')


Insertar la tabla location

In [32]:
df2[["barrio","distrito","latitude","latitude"]]

,barrio,distrito,latitude,latitude
0,Recoletos,Salamanca,"40,4221164","40,4221164"
1,Universidad-Malasaña,Centro,"40,422312","40,422312"
2,Goya,Salamanca,"40,4243638","40,4243638"
3,Opañel,Carabanchel,40.3891,40.3891
4,Rejas,San Blas,40.4420421,40.4420421
...,...,...,...,...
3015,Portazgo,Puente de Vallecas,40.3837117,40.3837117
3016,Pueblo Nuevo,Ciudad Lineal,40.4263335,40.4263335
3017,Valdemarín,Moncloa-Aravaca,40.4653434,40.4653434
3018,Trafalgar,Chamberí,40.4355968,40.4355968


In [58]:
df_location = df2[["barrio","distrito","latitude","longitude"]]
df_location

,barrio,distrito,latitude,longitude
0,Recoletos,Salamanca,"40,4221164","-3,6831486"
1,Universidad-Malasaña,Centro,"40,422312","-3,706137"
2,Goya,Salamanca,"40,4243638","-3,6705767"
3,Opañel,Carabanchel,40.3891,-3.7199
4,Rejas,San Blas,40.4420421,-3.5759197
...,...,...,...,...
3015,Portazgo,Puente de Vallecas,40.3837117,-3.6504517
3016,Pueblo Nuevo,Ciudad Lineal,40.4263335,-3.6382045
3017,Valdemarín,Moncloa-Aravaca,40.4653434,-3.7629673
3018,Trafalgar,Chamberí,40.4355968,-3.7004786


In [59]:
columnas = df_location.columns.to_list()
columnas

['barrio', 'distrito', 'latitude', 'longitude']

In [60]:
# Convertir DataFrame a lista de tuplas
location = [
    tuple(row[col] for col in columnas)
    for _, row in df_location.iterrows()
]

In [61]:
location

[('Recoletos', 'Salamanca', '40,4221164', '-3,6831486'),
 ('Universidad-Malasaña', 'Centro', '40,422312', '-3,706137'),
 ('Goya', 'Salamanca', '40,4243638', '-3,6705767'),
 ('Opañel', 'Carabanchel', '40.3891', '-3.7199'),
 ('Rejas', 'San Blas', '40.4420421', '-3.5759197'),
 ('Lista', 'Salamanca', '40.4295857', '-3.6735977'),
 ('Guindalera', 'Salamanca', '40.4390839', '-3.6613366'),
 ('Embajadores-Lavapiés', 'Centro', '40.4129747', '-3.7059398'),
 ('Lucero', 'Latina', '40.4025977', '-3.7530822'),
 ('Goya', 'Salamanca', '40.4241', '-3.67855'),
 ('Recoletos', 'Salamanca', '40.4226281', '-3.6831171'),
 ('El Plantío', 'Moncloa-Aravaca', '40.4767', '-3.82927'),
 ('Castellana', 'Salamanca', '40.4277347', '-3.685881'),
 ('Las Águilas', 'Latina', '40.377671', '-3.7731945'),
 ('Lista', 'Salamanca', '40.4318763', '-3.6736524'),
 ('Palacio', 'Centro', '40.4118684', '-3.7176348'),
 ('El Cañaveral', 'Vicálvaro', '40.411542', '-3.5447233'),
 ('Lista', 'Salamanca', '40.4334893', '-3.6773472'),
 ('Guin

In [64]:
df_location = df_location.where(pd.notnull(df_location), None)
location = df_location[columnas].values.tolist()

In [ ]:
df_location

,barrio,distrito,latitude,latitude
0,Recoletos,Salamanca,"40,4221164","40,4221164"
1,Universidad-Malasaña,Centro,"40,422312","40,422312"
2,Goya,Salamanca,"40,4243638","40,4243638"
3,Opañel,Carabanchel,40.3891,40.3891
4,Rejas,San Blas,40.4420421,40.4420421
...,...,...,...,...
3015,Portazgo,Puente de Vallecas,40.3837117,40.3837117
3016,Pueblo Nuevo,Ciudad Lineal,40.4263335,40.4263335
3017,Valdemarín,Moncloa-Aravaca,40.4653434,40.4653434
3018,Trafalgar,Chamberí,40.4355968,40.4355968


In [70]:
df_location.columns

Index(['barrio', 'distrito', 'latitude', 'longitude'], dtype='object')

Es importante que los datos estén en el formato adecuado para poder cargar la base de datos, tampoco podemos tener valores nan

In [79]:
# Reemplazar comas por puntos para convertir a float
df_location['latitude'] = df_location['latitude'].astype(str).str.replace(',', '.').astype(float)
df_location['longitude'] = df_location['longitude'].astype(str).str.replace(',', '.').astype(float)


In [80]:
datos_location = [
    tuple(row[col] if pd.notna(row[col]) else None for col in columnas)
    for _, row in df_location.iterrows()
]

In [81]:
datos_location

[('Recoletos', 'Salamanca', 40.4221164, -3.6831486),
 ('Universidad-Malasaña', 'Centro', 40.422312, -3.706137),
 ('Goya', 'Salamanca', 40.4243638, -3.6705767),
 ('Opañel', 'Carabanchel', 40.3891, -3.7199),
 ('Rejas', 'San Blas', 40.4420421, -3.5759197),
 ('Lista', 'Salamanca', 40.4295857, -3.6735977),
 ('Guindalera', 'Salamanca', 40.4390839, -3.6613366),
 ('Embajadores-Lavapiés', 'Centro', 40.4129747, -3.7059398),
 ('Lucero', 'Latina', 40.4025977, -3.7530822),
 ('Goya', 'Salamanca', 40.4241, -3.67855),
 ('Recoletos', 'Salamanca', 40.4226281, -3.6831171),
 ('El Plantío', 'Moncloa-Aravaca', 40.4767, -3.82927),
 ('Castellana', 'Salamanca', 40.4277347, -3.685881),
 ('Las Águilas', 'Latina', 40.377671, -3.7731945),
 ('Lista', 'Salamanca', 40.4318763, -3.6736524),
 ('Palacio', 'Centro', 40.4118684, -3.7176348),
 ('El Cañaveral', 'Vicálvaro', 40.411542, -3.5447233),
 ('Lista', 'Salamanca', 40.4334893, -3.6773472),
 ('Guindalera', 'Salamanca', 40.434233, -3.6672526),
 ('Guindalera', 'Salamanca

In [ ]:
# Verifica longitud
for fila in datos_location[:5]:
    print(fila, len(fila))  # Cada fila debe tener 4 elementos

('Recoletos', 'Salamanca', '40,4221164', '-3,6831486') 4
('Universidad-Malasaña', 'Centro', '40,422312', '-3,706137') 4
('Goya', 'Salamanca', '40,4243638', '-3,6705767') 4
('Opañel', 'Carabanchel', '40.3891', '-3.7199') 4
('Rejas', 'San Blas', '40.4420421', '-3.5759197') 4


In [83]:
sql_location = """
    INSERT IGNORE INTO location (barrio, distrito, latitude, longitude)
    VALUES (%s, %s, %s, %s)
"""
cursor2.executemany(sql_location, datos_location)
conn2.commit()

Se seguirá la misma estrategia de trabajo expuesta para cargar la base de datos cuando los datos extraidos esten limpios, teniendo en cuenta además que son dos fichero, uno para ventas y otro para alquileres